In [2]:
%matplotlib inline

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
import scipy.io
import time
import h5py
import bz2
import multiprocessing as mp


from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from numpy import linalg as LA
from scipy.spatial import Delaunay
from sklearn.neighbors import NearestNeighbors, KDTree

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef

#sys.path.insert(0, "../")
from info3d import *
from nn_matchers import *

In [3]:
with open('point_collection/new_contiguous_point_collection.pickle','rb') as f: 
    new_contiguous_point_collection = pickle.load(f)
    
with open('descriptors/new_complete_res5_4by5_descriptors.pickle','rb') as f:
    descriptors = pickle.load(f)
    
with open('descriptors/new_complete_RANSAC_res5_4by5_descriptors.pickle','rb') as f:
    ransac_descriptors = pickle.load(f)

In [ ]:
descriptors = ransac_descriptors

skip = 10

t0 = time.time()

for radius in np.arange(0.5,2.1,0.5):
    
    t1 = time.time()
    t2 = t1
    
    with open('successive_sample_points_per_release.pickle','rb') as f:
        successive_sample_points_per_release = pickle.load(f)
        
    try:
        with open('testing_results/successive/radius_{}_RANSAC_successive_scores.pickle'.format(radius), 'rb') as f:
            g_successive_scores = pickle.load(f)
            
        with open('testing_results/successive/radius_{}_RANSAC_successive_errors.pickle'.format(radius), 'rb') as f:
            g_successive_errors = pickle.load(f)
            
            
        if len(g_successive_scores) != len(g_successive_errors): continue
            
        print("Done with",5*len(g_successive_scores),"releases for radius",radius)
        
    except:

        g_successive_scores = []
        g_successive_errors = []
        
        print("New successive scores for radius",radius)

    for i in np.arange(1,100,skip)[len(g_successive_scores):]: # releases
        
        release_scores = []
        
        g_release_scores = []
        
        count = 0
        
        for k, successive_sample_points_per_release_per_obj in enumerate(successive_sample_points_per_release[i]):
    
            #print("  ",len(successive_sample_points_per_release_per_obj),"releases")
            growing_point_cloud = []
            
            new_vX = []
            new_vZ = []
            
            ransac_pointCloud = []
                
            for [obj_, object_name, original_vertex], reference_ransac in successive_sample_points_per_release_per_obj[:i]:

                try:
                    if len(ransac_pointCloud) == 0: # if empty, open. This only happens at beginning
                        with open("../ransac_pc/ransac_point_collection_{}.pickle".format(reference_ransac),'rb') as f:
                            ransac_trial_point_collection = pickle.load(f)

                        object_, ransac_pointCloud, tri_ = ransac_trial_point_collection[int(obj_)]
                        
                        ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(ransac_pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])

                except:
                    print("Can't get ransac samples for",i,obj_, object_name)
                    continue

                dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
                pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0]<=radius)[0]]]

                if len(pointCloud) == 0: continue
                    
                #Regular Accumulation
                if len(growing_point_cloud) == 0:
                    growing_point_cloud = pointCloud

                else:
                    growing_point_cloud = np.concatenate(
                        (growing_point_cloud,pointCloud),
                        axis=0
                    )
                    
                count += 1
                    
            if len(growing_point_cloud) == 0: 
                print("  No pc for release",i,"obj",k,"with name",object_name)
                continue
                    
            #Regular Processing
            growing_point_cloud = np.unique(growing_point_cloud,axis=0)
            
            try:
                p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                    growing_point_cloud,
                    down_resolution = 5,
                    cylindrical_quantization = [4,5]
                )
            except:
                p_descriptors = []
                p_keypoints = []

            # Resetting the diff_Ratio matrix
            diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
            diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
            diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

            local_keypoint_matches = []

            for i_r, ref_descriptor in enumerate(descriptors):

                r_descriptors = ref_descriptor[1]
                r_keypoints = ref_descriptor[2]

                matching_range = np.arange(r_descriptors.shape[1])

                try:    
                    f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                    diff = diff/np.amax(diff) # max-normalization of differences
                    diff_ratio = diff[:,0]/diff[:,1]
                    diff_ratios[:,i_r] = diff_ratio
                    diff_scores[:,i_r] = diff
                    diff_indexs[:,i_r] = f_nearestneighbor

                    # Taking note of the matched keypoints
                    local_keypoint_matches.append([
                        [obj_, object_name, original_vertex],
                        p_keypoints,
                        r_keypoints[f_nearestneighbor[:,0]]
                    ])

                except Exception as ex:
                    print("Error Matching:",ex)

            # Accumulating the diff_ratio matrix for every partial (rotated) object
            g_release_scores.append([
                [obj_, object_name, original_vertex],
                diff_ratios,
                diff_indexs,
                diff_scores,
                local_keypoint_matches
            ])

            if count % 33 == 0:
                #print('Test')
                print("   radius = {}: Done with {} releases ({}) ({} samples). Time to match {:.3f} seconds. ({})".format(
                    radius,
                    i,
                    k,
                    count,
                    time.time()-t2,
                    growing_point_cloud.shape
                )
                     )
                t2 = time.time()

                current_errors = NN_matcher(g_release_scores)
                print("   Error Rate:",np.sum(current_errors[:,1]/len(g_release_scores)))
            
        g_release_errors = NN_matcher(g_release_scores)
        
        try:
            print(radius,i,"Error Rate:",np.sum(g_release_errors[:,1]/len(g_release_errors)))
            #print(radius,i,"G_Error Rate:",np.sum(g_iteration_errors[:,1]/len(g_iteration_scores)))
        except:
            pass
        
        g_successive_scores.append([
            i,
            g_release_scores
        ])
        
        g_successive_errors.append([
            i,
            g_release_errors
        ])
    
        with open('testing_results/successive/radius_{}_RANSAC_successive_scores.pickle'.format(radius), 'wb') as f:
            pickle.dump(g_successive_scores,f)

        with open('testing_results/successive/radius_{}_RANSAC_successive_errors.pickle'.format(radius), 'wb') as f:
            pickle.dump(g_successive_errors,f)
            
        if i % 40  == 1:
            print("  Done with release {}, radius {} ({} actual pcs) in {:.3f} seconds".format(i,radius,count,time.time()-t1))
            t1 = time.time()

        
    print(" Done with generalized submap generation for radius {} in {:.3f} seconds".format(radius,time.time()-t0))
    t0 = time.time()
                

New successive scores for radius 0.5
   radius = 0.5: Done with 1 releases (32) (33 samples). Time to match 66.820 seconds. ((102, 6))
   Error Rate: 0.9090909090909092


/srv/jaybie/archive/spatial-privacy-master/nn_matchers.py:667: FutureWarning: in the future insert will treat boolean arrays and array-likes as a boolean index instead of casting it to integer
  good_matches_kp_idx = np.insert(good_matches[good_matches_ref_kp],True,good_matches_ref_kp)


  No pc for release 1 obj 44 with name Reception-Data61-L5.obj
   radius = 0.5: Done with 1 releases (66) (66 samples). Time to match 71.352 seconds. ((150, 6))
   Error Rate: 0.8484848484848484


/srv/jaybie/archive/spatial-privacy-master/nn_matchers.py:667: FutureWarning: in the future insert will treat boolean arrays and array-likes as a boolean index instead of casting it to integer
  good_matches_kp_idx = np.insert(good_matches[good_matches_ref_kp],True,good_matches_ref_kp)
/home/jaybie/.conda/envs/3d_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jaybie/.conda/envs/3d_env/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


   radius = 0.5: Done with 1 releases (99) (99 samples). Time to match 63.259 seconds. ((113, 6))
   Error Rate: 0.8686868686868685
0.5 1 Error Rate: 0.8686868686868685
  Done with release 1, radius 0.5 (99 actual pcs) in 201.521 seconds


...

0.5 31 Error Rate: 0.35000000000000003
   radius = 0.5: Done with 41 releases (32) (1353 samples). Time to match 2888.478 seconds. ((10449, 6))


/srv/jaybie/archive/spatial-privacy-master/nn_matchers.py:667: FutureWarning: in the future insert will treat boolean arrays and array-likes as a boolean index instead of casting it to integer
  good_matches_kp_idx = np.insert(good_matches[good_matches_ref_kp],True,good_matches_ref_kp)


   Error Rate: 0.09090909090909091
   radius = 0.5: Done with 41 releases (61) (2541 samples). Time to match 2416.240 seconds. ((7019, 6))


/srv/jaybie/archive/spatial-privacy-master/nn_matchers.py:667: FutureWarning: in the future insert will treat boolean arrays and array-likes as a boolean index instead of casting it to integer
  good_matches_kp_idx = np.insert(good_matches[good_matches_ref_kp],True,good_matches_ref_kp)


   Error Rate: 0.16129032258064513
